# Linear programming example

Created by Alex Dowling (alexdowling.net) while at the University of Wisconsin-Madison

## Define Problem

In [1]:
include("../DegeneracyHunter.jl")
using JuMP

function lp1()
	m = Model()

	@variable(m, 0 <= x[1:3] <= 5)
	@variable(m, 0 <= y <= 0)
	@constraint(m, x[1] + x[2] >= 1)
	@constraint(m, x[1] + x[2] + x[3] == 1)
	@constraint(m, x[2] - 2*x[3] <= 1)
	@constraint(m, x[1] + x[3] >= 1)
	@constraint(m, x[1] + x[2] + x[3] == 1) # Redundant constraint - makes problem degenerate
	@objective(m, Min, sum{x[i],i=1:3})
	
	return m
end

function initialize!(m::Model)
	x = getvariable(m,:x)
	setvalue(x[1], 1.0)
	setvalue(x[2], 5.0)
	setvalue(x[3], -1.0)
	
	# Intentionally do not initialize y
	
	return nothing
end

initialize! (generic function with 1 method)

## Create and initialize model

In [2]:
m = lp1()
initialize!(m)
print(m)

Min x[1] + x[2] + x[3]
Subject to
 x[1] + x[2] ≥ 1
 x[1] + x[2] + x[3] = 1
 x[2] - 2 x[3] ≤ 1
 x[1] + x[3] ≥ 1
 x[1] + x[2] + x[3] = 1
 0 ≤ x[i] ≤ 5 ∀ i ∈ {1,2,3}
 0 ≤ y ≤ 0


## Check initialization

### Check if initial values are within variable bounds

In [3]:
DegeneracyHunter.printVariableDiagnostics(m)

Uninitialized Variables: 
y
 
Variable Violating Bounds: 
x[3] = -1.0, bounds: [ 0.0 , 5.0 ]


### Print equations with absolute residuals greater than 0.001

In [4]:
DegeneracyHunter.printInfeasibleEquations(m, 0.001)

Infeasible equations: 
r[4] = -1.0
x[1] + x[3] ≥ 1
x[1] = x[1] = 1.0  	 [0.0,5.0]
x[3] = x[3] = -1.0  	 [0.0,5.0]
 
r[2] = 4.0
x[1] + x[2] + x[3] = 1
x[1] = x[1] = 1.0  	 [0.0,5.0]
x[2] = x[2] = 5.0  	 [0.0,5.0]
x[3] = x[3] = -1.0  	 [0.0,5.0]
 
r[5] = 4.0
x[1] + x[2] + x[3] = 1
x[1] = x[1] = 1.0  	 [0.0,5.0]
x[2] = x[2] = 5.0  	 [0.0,5.0]
x[3] = x[3] = -1.0  	 [0.0,5.0]
 
r[3] = 6.0
x[2] - 2 x[3] ≤ 1
x[2] = x[2] = 5.0  	 [0.0,5.0]
x[3] = x[3] = -1.0  	 [0.0,5.0]
 


### Check for degenerate constraints

In [5]:
DegeneracyHunter.degeneracyHunter(m, includeBounds=true);

******************************************
Welcome to Degeneracy Hunter!
 
Options: 
includeBounds = true
includeWeaklyActive = true
removeFixedVar = true
epsiActive = 1.0e-6
epsiLambda = 1.0e-6
lambdaM = 100000.0
 
 
Smallest non-zero element in Jacobian = 1.0
Variable: 
x[1] = x[1]
Equation: 
x[1] + x[2] ≥ 1
 
Largest non-zero element in Jacobian = -2.0
Variable: 
x[3] = x[3]
Equation: 
x[2] - 2 x[3] ≤ 1
 
 
(Weakly) Active Inequality Constraints: 0
Inactive Inequality Constraints: 3
Equality Constraints: 2
 
Variables: 4
(Weakly) Active Variable Bounds: 4
Fixed Variables: 1
 
Adding Jacobian elements for bounds... 3.62e-6 seconds
Assembling J_sparse... 4.866e-6 seconds
Assembling J_active... 0.06035891 seconds
Identified 2 candidate constraints/bounds.
Degenerate set from candidate search:
IDS 0...
y = 1.0000, l = -1.0000  	 x[1] + x[2] ≥ 1
y = 1.0000, l = 1.0000  	 x[1] + x[2] + x[3] = 1
 
Involved variables: 
x[1] = x[1] = 0.0  	 [0.0,5.0]
x[2] = x[2] = 0.0  	 [0.0,5.0]
x[3] = x[3

## Solve model

In [6]:
tic()
status = solve(m)
tm = toq()

0.041945466

## Analyze solution

### Print inactive constraints at solution

In [7]:
DegeneracyHunter.printInactiveEquations(m)

Inactive Equations: 
x[2] - 2 x[3] ≤ 1


### Package and print problem size information

In [8]:
ps = DegeneracyHunter.assembleProblemStats(m,status,tm)
DegeneracyHunter.printProblemStats(ps)

 
Continous Variables (total): 4
	 Fixed: 1
	 Free: 3
		 Lower Bounded Only: 0
		 Upper Bounded Only: 0
		 Lower & Upper Bounded: 3
		 Unbounded: 0
		 Active Bounds: 2
 
Binary Variable (total): 0
	 Fixed: 0
	 Free: 0
 
Linear Constraints: 5
	 Equality: 2
	 Ineq. (Active): 2
	 Ineq. (Inactive): 1
 
Quadratic Constraints: 0
	 Equality: 0
	 Ineq. (Active): 0
	 Ineq. (Inactive): 0
 
Nonlinear Constraints: 0
	 Equality: 0
	 Ineq. (Active): 0
	 Ineq. (Inactive): 0
 
Solver Exit Status: Optimal
Solve Time: 0.041945466s
 
